In [20]:
import pandas as pd
import re
import math
import os
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [22]:
# Define the directory paths
directories = [
    ("..\\new_dataset\\eclipse"),
    #("..\\new_dataset\\firefox", "..\\BOW_dataset\\firefox"),
    #("..\\new_dataset\\netbeans", "..\\BOW_dataset\\netbeans"),
    #("..\\new_dataset\\openoffice", "..\\BOW_dataset\\openoffice")
]


In [23]:
# Function to preprocess bug description text
def preprocess_bug_description(description):

    # convert description to string
    description = str(description)
    
    # Convert to lowercase
    description = description.lower()

    # Remove punctuation and special characters
    description = re.sub(r'[^a-zA-Z0-9\s]', '', description)

    # Remove numbers
    description = re.sub(r'\b\d+\b', '', description)

    return description

In [24]:
def get_bug_description(bug_id, file_data):        
    return file_data[file_data['bug_id'] == bug_id]['description'].values[0]

In [ ]:
def get_unique_words(first_sentence, second_sentence):
    # Split the sentences into individual words
    first_words = set(first_sentence.split(" "))
    second_words = set(second_sentence.split(" "))
    
    # Remove common duplicate words by taking the union of the sets
    total_words = first_words.union(second_words)
    
    return total_words

In [25]:
# Iterate over each directory
for source_dir in directories:
    # Iterate over each file in the directory
    for file_name in os.listdir(source_dir):
        # Check if the file is a CSV file and contains "_new_similarity" in its name
        if file_name.endswith(".csv") and "_pairs" in file_name:
            # Load the CSV file with similarity data
            pairs_data = pd.read_csv(os.path.join(source_dir, file_name))
            print("Pairs file loaded:", file_name)

            # Load the original bug reports file
            original_file_name = file_name.replace("_pairs.csv", "_new.csv")
            bug_reports_file = pd.read_csv(os.path.join(source_dir, original_file_name))
            print("Original bug reports file loaded:", original_file_name)

            # iterate over each row of the pairs data and get the issue_id and duplicate columns
            for index, row in pairs_data.iterrows():
                issue_id = row['issue_id']
                duplicates = row['duplicate']

                # check if dulicates have ; in it so split it based on ;
                if ";" in duplicates:
                    duplicates = duplicates.split(";")
                else:
                    duplicates = [duplicates]
                
                # get the bug description of the issue_id
                bug_description = get_bug_description(issue_id, bug_reports_file)
                bug_description = preprocess_bug_description(bug_description)
                print("Bug description:", bug_description)

                # get the bug description of the duplicates
                for duplicate in duplicates:
                    duplicate = int(duplicate)
                    duplicate_description = get_bug_description(duplicate, bug_reports_file)
                    duplicate_description = preprocess_bug_description(duplicate_description)
                    print("Duplicate description:", duplicate_description)

                # Defie a set to store all unique words in all the bug descriptions
                unique_words = set()
                unique_words = get_unique_words(bug_description, duplicate_description)
                print("Unique words:", unique_words)

                



Pairs file loaded: eclipse_small_pairs.csv
Original bug reports file loaded: eclipse_small_new.csv


{'21st', 'Science', 'learning', 'of', 'century', 'job', 'for', 'data', 'machine', 'key', 'Data', 'the', 'science', 'is', 'sexiest'}


In [27]:
wordDictA = dict.fromkeys(total, 0) 
wordDictB = dict.fromkeys(total, 0)
for word in first_sentence:
    wordDictA[word]+=1
    
for word in second_sentence:
    wordDictB[word]+=1

In [28]:
pd.DataFrame([wordDictA, wordDictB])


,21st,Science,learning,of,century,job,for,data,machine,key,Data,the,science,is,sexiest
0,1,1,0,1,1,1,0,0,0,0,1,2,0,1,1
1,0,0,1,0,0,0,1,1,1,1,0,1,1,1,0


In [29]:
def computeTF(wordDict, doc):
    tfDict = {}
    corpusCount = len(doc)
    for word, count in wordDict.items():
        tfDict[word] = count/float(corpusCount)
    return(tfDict)
#running our sentences through the tf function:
tfFirst = computeTF(wordDictA, first_sentence)
tfSecond = computeTF(wordDictB, second_sentence)
#Converting to dataframe for visualization
tf = pd.DataFrame([tfFirst, tfSecond])

In [30]:
filtered_sentence = []
for word in wordDictA:
    if str(word) not in set(stopwords.words('english')):
        filtered_sentence.append(word)

In [31]:
filtered_sentence


['21st',
 'Science',
 'learning',
 'century',
 'job',
 'data',
 'machine',
 'key',
 'Data',
 'science',
 'sexiest']

In [32]:
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / (float(val) + 1))
        
    return(idfDict)
#inputing our sentences in the log file
idfs = computeIDF([wordDictA, wordDictB])

In [33]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return(tfidf)
#running our two sentences through the IDF:
idfFirst = computeTFIDF(tfFirst, idfs)
idfSecond = computeTFIDF(tfSecond, idfs)
#putting it in a dataframe
tf_idf= pd.DataFrame([idfFirst, idfSecond])

In [34]:
tf_idf

,21st,Science,learning,of,century,job,for,data,machine,key,Data,the,science,is,sexiest
0,0.030103,0.030103,0.000000,0.030103,0.030103,0.030103,0.000000,0.000000,0.000000,0.000000,0.030103,0.060206,0.000000,0.030103,0.030103
1,0.000000,0.000000,0.037629,0.000000,0.000000,0.000000,0.037629,0.037629,0.037629,0.037629,0.000000,0.037629,0.037629,0.037629,0.000000
